In [5]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML, clear_output, Javascript
import ipywidgets as widgets
import io
import base64

# Create button to start webcam
start_button = widgets.Button(description='Start Webcam')
output = widgets.Output()

def on_start_clicked(b):
    clear_output(wait=True)
    print("Initializing webcam...")
    display_webcam_with_zoom()

start_button.on_click(on_start_clicked)
display(start_button, output)

def display_webcam_with_zoom():
    """Display webcam stream with zoom functionality"""

    # Create container for webcam display
    webcam_container = widgets.Output()
    display(webcam_container)

    # HTML for webcam view with zoom controls
    html_content = """
    <div style="margin: 20px auto; text-align: center;">
        <h3>Webcam Zoom Controller</h3>
        <div id="webcam-container" style="position: relative; width: 800px; height: 600px; margin: 0 auto;
                                         border: 2px solid #ddd; border-radius: 8px; overflow: hidden; background-color: #000;">
            <div id="video-container" style="position: absolute; width: 100%; height: 100%; transform-origin: 0 0;">
                <video id="webcam" autoplay playsinline style="width: 100%; height: 100%; display: block;"></video>
            </div>
            <div id="zoom-indicator" style="position: absolute; top: 10px; right: 10px; background-color: rgba(0, 0, 0, 0.5);
                                         color: white; padding: 5px 10px; border-radius: 4px; font-size: 16px;
                                         font-weight: bold; display: none;">1.0x</div>
        </div>

        <div style="margin-top: 15px; text-align: center;">
            <div style="display: inline-flex; align-items: center; gap: 10px; margin-bottom: 10px;">
                <button id="zoom-out" style="background-color: #2196F3; color: white; border: none;
                                           border-radius: 4px; padding: 5px 15px; cursor: pointer; font-size: 16px;">−</button>
                <div id="zoom-level" style="background-color: #fff; padding: 5px 10px; border-radius: 4px;
                                          border: 1px solid #ddd; min-width: 50px; text-align: center;">100%</div>
                <button id="zoom-in" style="background-color: #2196F3; color: white; border: none;
                                          border-radius: 4px; padding: 5px 15px; cursor: pointer; font-size: 16px;">+</button>
                <button id="zoom-reset" style="background-color: #f44336; color: white; border: none;
                                             border-radius: 4px; padding: 5px 15px; cursor: pointer; font-size: 16px;">Reset</button>
            </div>
        </div>

        <div style="margin-top: 15px; background-color: #f0f0f0; padding: 10px; border-radius: 5px; display: inline-block;">
            <b>Controls:</b>
            <ul style="text-align: left; margin-bottom: 0;">
                <li>Use trackpad pinch gestures or mouse wheel to zoom in/out</li>
                <li>Click and drag to pan when zoomed in</li>
                <li>Double-click to reset zoom</li>
            </ul>
        </div>
    </div>
    """

    # JavaScript for webcam access and zoom functionality
    js_code = """
    // Access webcam
    async function setupWebcam() {
        try {
            const stream = await navigator.mediaDevices.getUserMedia({
                video: {
                    width: { ideal: 1280 },
                    height: { ideal: 720 },
                    facingMode: "user"
                },
                audio: false
            });

            const videoElement = document.getElementById('webcam');
            videoElement.srcObject = stream;

            return new Promise((resolve) => {
                videoElement.onloadedmetadata = () => {
                    resolve(videoElement);
                };
            });
        } catch (error) {
            console.error('Error accessing webcam:', error);
            document.getElementById('webcam-container').innerHTML =
                '<div style="color: white; padding: 20px; text-align: center;">Error accessing webcam.<br>' +
                'Please make sure you have granted camera permissions and try again.</div>';
        }
    }

    // Initialize zoom functionality for webcam
    function initializeZoom() {
        const webcamContainer = document.getElementById('webcam-container');
        const videoContainer = document.getElementById('video-container');
        const zoomInBtn = document.getElementById('zoom-in');
        const zoomOutBtn = document.getElementById('zoom-out');
        const zoomResetBtn = document.getElementById('zoom-reset');
        const zoomLevelDisplay = document.getElementById('zoom-level');
        const zoomIndicator = document.getElementById('zoom-indicator');

        if (!webcamContainer || !videoContainer) {
            console.error('Required elements not found');
            return;
        }

        // Set initial values
        let scale = 1;
        let posX = 0;
        let posY = 0;
        let startX = 0;
        let startY = 0;
        let startPosX = 0;
        let startPosY = 0;
        let isPanning = false;
        const MIN_SCALE = 1;
        const MAX_SCALE = 100;
        const ZOOM_STEP = 0.2;

        // Timer for hiding zoom indicator
        let zoomIndicatorTimer = null;

        // Update the transform and displays
        function updateTransform() {
            videoContainer.style.transform = `translate(${posX}px, ${posY}px) scale(${scale})`;
            zoomLevelDisplay.textContent = `${Math.round(scale * 100)}%`;

            // Update the zoom indicator with a formatted multiplier
            const formattedZoom = scale.toFixed(1) + "x";
            zoomIndicator.textContent = formattedZoom;

            // Show the zoom indicator
            zoomIndicator.style.display = 'block';

            // Clear any existing timer
            if (zoomIndicatorTimer) {
                clearTimeout(zoomIndicatorTimer);
            }

            // Hide the indicator after 2 seconds of inactivity
            zoomIndicatorTimer = setTimeout(() => {
                if (scale === 1) {
                    zoomIndicator.style.display = 'none';
                }
            }, 2000);
        }

        // Reset zoom and position
        function resetZoomAndPosition() {
            scale = 1;
            posX = 0;
            posY = 0;
            updateTransform();
            zoomIndicator.style.display = 'none';
        }

        // Handle wheel event for zoom (including trackpad pinch gestures)
        webcamContainer.addEventListener('wheel', function(e) {
            e.preventDefault();

            // Get mouse position relative to container
            const rect = webcamContainer.getBoundingClientRect();
            const mouseX = e.clientX - rect.left;
            const mouseY = e.clientY - rect.top;

            // Detect if this is a pinch gesture (trackpad) or regular scroll
            // For trackpad pinch gestures on most platforms, ctrlKey is true
            const isPinch = e.ctrlKey;

            // Calculate zoom direction and factor
            let delta;
            if (isPinch) {
                // For pinch gestures, use deltaY but with different sensitivity
                delta = -e.deltaY * 0.01;
            } else {
                // For mouse wheel, use standard direction
                delta = e.deltaY < 0 ? 1 : -1;
            }

            const zoomFactor = 1 + (delta * ZOOM_STEP);
            const newScale = scale * zoomFactor;

            // Apply zoom constraints
            if (newScale < MIN_SCALE || newScale > MAX_SCALE) return;

            // Calculate new position to zoom at mouse point
            posX = mouseX - (mouseX - posX) * zoomFactor;
            posY = mouseY - (mouseY - posY) * zoomFactor;
            scale = newScale;

            updateTransform();
        });

        // Handle mouse down for panning
        webcamContainer.addEventListener('mousedown', function(e) {
            if (scale <= 1) return;

            e.preventDefault();
            startX = e.clientX;
            startY = e.clientY;
            startPosX = posX;
            startPosY = posY;
            isPanning = true;
            webcamContainer.style.cursor = 'grabbing';
        });

        // Handle mouse move for panning
        window.addEventListener('mousemove', function(e) {
            if (!isPanning) return;

            posX = startPosX + (e.clientX - startX);
            posY = startPosY + (e.clientY - startY);
            updateTransform();
        });

        // Handle mouse up to stop panning
        window.addEventListener('mouseup', function() {
            isPanning = false;
            webcamContainer.style.cursor = 'grab';
        });

        // Handle double click to reset
        webcamContainer.addEventListener('dblclick', resetZoomAndPosition);

        // Zoom buttons functionality
        zoomInBtn.addEventListener('click', function() {
            if (scale >= MAX_SCALE) return;

            const zoomFactor = 1 + ZOOM_STEP;
            const newScale = scale * zoomFactor;

            // Calculate center point zoom
            const containerWidth = webcamContainer.offsetWidth;
            const containerHeight = webcamContainer.offsetHeight;
            const centerX = containerWidth / 2;
            const centerY = containerHeight / 2;

            posX = centerX - (centerX - posX) * zoomFactor;
            posY = centerY - (centerY - posY) * zoomFactor;
            scale = newScale > MAX_SCALE ? MAX_SCALE : newScale;

            updateTransform();
        });

        zoomOutBtn.addEventListener('click', function() {
            if (scale <= MIN_SCALE) return;

            const zoomFactor = 1 / (1 + ZOOM_STEP);
            const newScale = scale * zoomFactor;

            // Calculate center point zoom
            const containerWidth = webcamContainer.offsetWidth;
            const containerHeight = webcamContainer.offsetHeight;
            const centerX = containerWidth / 2;
            const centerY = containerHeight / 2;

            posX = centerX - (centerX - posX) * zoomFactor;
            posY = centerY - (centerY - posY) * zoomFactor;
            scale = newScale < MIN_SCALE ? MIN_SCALE : newScale;

            updateTransform();
        });

        zoomResetBtn.addEventListener('click', resetZoomAndPosition);

        // Set initial cursor
        webcamContainer.style.cursor = 'grab';

        // Add touch support for mobile devices
        let lastTouchDistance = 0;

        webcamContainer.addEventListener('touchstart', function(e) {
            if (e.touches.length === 2) {
                // Store initial touch positions for pinch-zoom
                const touch1 = e.touches[0];
                const touch2 = e.touches[1];
                lastTouchDistance = Math.hypot(
                    touch2.clientX - touch1.clientX,
                    touch2.clientY - touch1.clientY
                );
            } else if (e.touches.length === 1 && scale > 1) {
                // Start panning with one finger if zoomed in
                const touch = e.touches[0];
                startX = touch.clientX;
                startY = touch.clientY;
                startPosX = posX;
                startPosY = posY;
                isPanning = true;
            }
        });

        webcamContainer.addEventListener('touchmove', function(e) {
            e.preventDefault();

            if (e.touches.length === 2) {
                // Handle pinch-zoom
                const touch1 = e.touches[0];
                const touch2 = e.touches[1];
                const currentDistance = Math.hypot(
                    touch2.clientX - touch1.clientX,
                    touch2.clientY - touch1.clientY
                );

                if (lastTouchDistance > 0) {
                    const touchDelta = currentDistance - lastTouchDistance;
                    const zoomFactor = touchDelta > 0 ? 1.02 : 0.98;
                    const newScale = scale * zoomFactor;

                    if (newScale >= MIN_SCALE && newScale <= MAX_SCALE) {
                        // Calculate center of pinch
                        const centerX = (touch1.clientX + touch2.clientX) / 2 - webcamContainer.getBoundingClientRect().left;
                        const centerY = (touch1.clientY + touch2.clientY) / 2 - webcamContainer.getBoundingClientRect().top;

                        posX = centerX - (centerX - posX) * zoomFactor;
                        posY = centerY - (centerY - posY) * zoomFactor;
                        scale = newScale;

                        updateTransform();
                    }
                }

                lastTouchDistance = currentDistance;

            } else if (e.touches.length === 1 && isPanning) {
                // Handle panning with one finger
                const touch = e.touches[0];
                posX = startPosX + (touch.clientX - startX);
                posY = startPosY + (touch.clientY - startY);
                updateTransform();
            }
        });

        webcamContainer.addEventListener('touchend', function() {
            isPanning = false;
            lastTouchDistance = 0;
        });
    }

    // Initialize webcam and zoom functionality
    (async function() {
        await setupWebcam();
        initializeZoom();
        console.log('Webcam and zoom initialized');
    })();
    """

    with webcam_container:
        display(HTML(html_content))
        display(Javascript(js_code))

    print("Webcam initialized successfully.")
    print("You can now use trackpad pinch gestures to zoom in/out of the webcam stream.")
    print("Click and drag to pan when zoomed in, and double-click to reset the view.")
    print("Maximum zoom level has been increased to 50x.")
    print("A zoom level indicator will appear when zooming.")

# Display initial instructions
print("Click the 'Start Webcam' button to enable your camera.")
print("After starting, you'll be asked for permission to access your webcam.")
print("Once permitted, you can use trackpad pinch gestures to zoom in/out without losing quality.")
print("This enhanced version supports up to 50x zoom with a real-time zoom level indicator.")

Button(description='Start Webcam', style=ButtonStyle())

Output()

Click the 'Start Webcam' button to enable your camera.
After starting, you'll be asked for permission to access your webcam.
Once permitted, you can use trackpad pinch gestures to zoom in/out without losing quality.
This enhanced version supports up to 50x zoom with a real-time zoom level indicator.
